<a href="https://colab.research.google.com/github/tiasnarmi/acadamic/blob/master/HR_Analytics_v0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
%matplotlib inline
import seaborn as sns
#import statsmodels.api as sm
import scipy.stats as stats
from sklearn.preprocessing import LabelEncoder
import copy
import io

sns.set() #setting the default seaborn style for our plots

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
train_df = pd.read_csv('/content/drive/My Drive/Analytics Vidhya/HR Analytics/train_LZdllcl.csv')
test_df = pd.read_csv('/content/drive/My Drive/Analytics Vidhya/HR Analytics/test_2umaH9m.csv')

In [9]:
train_df.head()

,employee_id,department,region,education,gender,recruitment_channel,no_of_trainings,age,previous_year_rating,length_of_service,KPIs_met >80%,awards_won?,avg_training_score,is_promoted
0,65438,Sales & Marketing,region_7,Master's & above,f,sourcing,1,35,5.0,8,1,0,49,0
1,65141,Operations,region_22,Bachelor's,m,other,1,30,5.0,4,0,0,60,0
2,7513,Sales & Marketing,region_19,Bachelor's,m,sourcing,1,34,3.0,7,0,0,50,0
3,2542,Sales & Marketing,region_23,Bachelor's,m,other,2,39,1.0,10,0,0,50,0
4,48945,Technology,region_26,Bachelor's,m,other,1,45,3.0,2,0,0,73,0


In [10]:
print (train_df.shape)
print (test_df.shape)

(54808, 14)
(23490, 13)


In [0]:
# indexes_train = train_df[( train_df['is_promoted'] == 1 ) & (train_df['awards_won?'] == 1) & (train_df['department'] == 'R&D')].index
# train_df.drop(indexes_train, inplace=True)

In [0]:
train_emp_id = train_df['employee_id']
test_emp_id = test_df['employee_id']

train_df.drop(['employee_id'], inplace=True, axis=1)
test_df.drop(['employee_id'], inplace=True, axis=1)

In [0]:

train_df.drop(train_df.columns.difference(['awards_won?','avg_training_score','department','is_promoted']), 1, inplace=True)
test_df.drop(test_df.columns.difference(['awards_won?','avg_training_score','department']), 1, inplace=True)


In [0]:
train_dfmerged = pd.concat([train_df[['awards_won?'
                                        ,'avg_training_score']],pd.get_dummies(train_df['department'],drop_first = True)],axis=1)

test_dfmerged = pd.concat([test_df[['awards_won?'
                                        ,'avg_training_score']],pd.get_dummies(test_df['department'],drop_first = True)],axis=1)

In [15]:
print (train_dfmerged.shape)
print (test_dfmerged.shape)

(54808, 10)
(23490, 10)


In [0]:
from sklearn.model_selection import train_test_split

y = train_df['is_promoted']
X = train_dfmerged 
 

seed = 2
test_size = 0.2
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=seed)

In [0]:
from xgboost import XGBClassifier
###Train the model with the best params
modelXg = XGBClassifier(learning_rate=0.1, n_estimators=80, max_depth=4, min_child_weight=7, 
                     gamma=0.4,nthread=4, subsample=0.8, colsample_bytree=0.8, objective= 'binary:logistic',scale_pos_weight=3,seed=29)

# modelXg = XGBClassifier(objective= 'binary:logistic',seed=29)

modelXg.fit(X_train, y_train)
y_xg = modelXg.predict(X_test)

In [18]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix

## Determine whether your model is overfitting or not , with the help of ROC.
print(confusion_matrix(y_test, y_xg))
predictions = [value for value in y_xg]
accuracy = accuracy_score(y_test, predictions)
precision = precision_score(y_test, predictions)
recall = recall_score(y_test, predictions)
f1 = f1_score(y_test, predictions)
print("Accuracy_score: %.2f%% on test dataset" % (accuracy * 100.0))
print("precision_score: %.2f%% on test dataset" % (precision * 100.0))
print("recall_score: %.2f%% on test dataset" % (recall * 100.0))
print("f1_score: %.2f%% on test dataset" % (f1 * 100.0))
print("roc_auc test set", roc_auc_score(y_test, modelXg.predict_proba(X_test)[:,1]))
print("roc_auc training set", roc_auc_score(y_train, modelXg.predict_proba(X_train)[:,1]))

[[9941   74]
 [ 599  348]]
Accuracy_score: 93.86% on test dataset
precision_score: 82.46% on test dataset
recall_score: 36.75% on test dataset
f1_score: 50.84% on test dataset
roc_auc test set 0.7361533728973594
roc_auc training set 0.7539139898915057


In [0]:
submission = pd.DataFrame(columns=['employee_id','is_promoted'])

In [20]:
y_valid = modelXg.predict(test_dfmerged)
#submission = test_dfmerged.copy()
submission['employee_id'] = test_emp_id
submission['is_promoted'] = y_valid
submission.to_csv("solution.csv",index=False)
submission['is_promoted'].value_counts()

0    22632
1      858
Name: is_promoted, dtype: int64